In [1]:
# Imports
import pandas as pd
import oracledb

In [2]:
dsn = oracledb.makedsn("localhost", 1522, service_name="stu")
connection = oracledb.connect(user="ora_msan99", password="a85263259", dsn=dsn)

Province=[]
cur = connection.cursor()
for row in cur.execute("select * from Province"):
    Province.append(row)

cur.close()
Industry=[]
cur = connection.cursor()
for row in cur.execute("select * from Industry"):
    Industry.append(row)

    
cur.close()

IndustryMapping=[]

cur = connection.cursor()
for row in cur.execute("select * from IndustryMapping"):
    IndustryMapping.append(row)
    
cur.close()
Emissions=[]
cur = connection.cursor()
for row in cur.execute("select * from Emissions"):
    Emissions.append(row)
    
cur.close()

In [11]:
emissions = pd.DataFrame(Emissions,columns=['province','IndustryName', 'IOIC', 'year',  'Emissions', 'industry_code'])

industry = pd.DataFrame(IndustryMapping,columns=['IndustryCode', 'IndustryName', 'NAICS', 'IOIC', 'Durable'])

population = pd.DataFrame(Province,columns=['province', 'year', 'Population'])

employment = pd.DataFrame(Industry,columns=['Good_Type', 'province', 'year','No_Employed'])

In [12]:
# model1: emissions/employed (by year, province) ~ good type(dummy) + province (dummy) + year

# joining emissions and industry together
emissions_industry = pd.merge(emissions, industry, left_on="industry_code", right_on="IndustryCode")

# filter out canada in emissions_industry dataset
emissions_industry = emissions_industry[emissions_industry["province"] != "Canada"]

# mutate a new column called ProvinceYearDurable in EmissionsIndustry
emissions_industry["ProvinceYearDurable"] = emissions_industry["province"] + emissions_industry["year"].astype(str) + emissions_industry["Durable"].astype(str)

# drop a few duplicate columns
# emissions_industry.drop(

In [14]:
emissions_industry

,province,IndustryName_x,IOIC_x,year,Emissions,industry_code,IndustryCode,IndustryName_y,NAICS,IOIC_y,Durable,ProvinceYearDurable
0,Alberta,Households: Motor fuels and lubricants,9999999,2013,11184,99999,99999,Households: Motor fuels and lubricants,999,9999999,0,Alberta20130
1,Alberta,Households: Motor fuels and lubricants,9999999,2014,11411,99999,99999,Households: Motor fuels and lubricants,999,9999999,0,Alberta20140
2,Alberta,Households: Motor fuels and lubricants,9999999,2015,10480,99999,99999,Households: Motor fuels and lubricants,999,9999999,0,Alberta20150
3,Alberta,Households: Motor fuels and lubricants,9999999,2016,10337,99999,99999,Households: Motor fuels and lubricants,999,9999999,0,Alberta20160
4,Alberta,Households: Motor fuels and lubricants,9999999,2017,10835,99999,99999,Households: Motor fuels and lubricants,999,9999999,0,Alberta20170
...,...,...,...,...,...,...,...,...,...,...,...,...
6155,Nunavut,Miscellaneous chemical product manufacturing,BS325C0,2018,1,325A,325A,Miscellaneous chemical product manufacturing,325,BS325C0,0,Nunavut20180
6156,Nunavut,Miscellaneous chemical product manufacturing,BS325C0,2019,1,325A,325A,Miscellaneous chemical product manufacturing,325,BS325C0,0,Nunavut20190
6157,Nunavut,Miscellaneous chemical product manufacturing,BS325C0,2020,1,325A,325A,Miscellaneous chemical product manufacturing,325,BS325C0,0,Nunavut20200
6158,Nunavut,Miscellaneous chemical product manufacturing,BS325C0,2021,1,325A,325A,Miscellaneous chemical product manufacturing,325,BS325C0,0,Nunavut20210


# linear regression: 

models:

1. emissions/population (by year, province) ~ province(dummy) * year ---> households
2. emissions/employed (by year, province) ~ good type(dummy) + province (dummy) + year

In [ ]:
# filter out canada in population dataset
population = population[population["Geography"] != "Canada"]

# mutate a new column called ProvinceYear in Population
population["ProvinceYear"] = population["Geography"] + population["year"].astype(str)

# joining emissions+industry with population dataset
emissions_industry_population_data = pd.merge(emissions_industry, population, on="ProvinceYear")

# clean up data
model1_data = emissions_industry_population_data.drop(columns=["year_y", "Geography", "IndustryCode", "IndustryName", "IOIC", ])

In [ ]:
# filter employment data to stop using Canada and Manufacturing
employment_tidy = employment[(employment["province"] != "Canada") & (employment["naics_industry"] != "Manufacturing")]

# rename naics_industry column, redesign column to be called durable in employment data that is 0 or 1
employment_tidy['durable'] = employment_tidy['naics_industry'].map({'Durables': 1, 'Non-durables': 0})
employment_tidy = employment_tidy.drop(columns=['naics_industry'])

# mutate employment data to create province+year+durable
employment_tidy["ProvinceYearDurable"] = employment_tidy["province"] + employment_tidy["year"].astype(str) + employment_tidy["durable"].astype(str)

# join emissions_industry and employment data
emissions_industry_employment = pd.merge(emissions_industry, employment_tidy, on="ProvinceYearDurable")

# drop a few columns
emissions_industry_employment.drop(columns=["province_y", "year_y", "durable", "IndustryCode", "IndustryName", "IOIC"], inplace=True)

# rename some columns
emissions_industry_employment_rename_dict = {'year_x': "year", "count_x": "emissions", "count_y": "employed"}
emissions_industry_employment.rename(columns=emissions_industry_employment_rename_dict, inplace=True)

# filter out household data to remove household 
emissions_industry_employment = emissions_industry_employment[~emissions_industry_employment["ioic_code"].isin(['9999999', '9999998'])]

# mutate employed to divide by how many industries there are per province; must make an assumption that each industry is equal in size. (not enough data to assume otherwise)
emissions_industry_employment["employed"] = emissions_industry_employment["employed"]/97*1000

# create response variable: 
emissions_industry_employment[emissions_per_employed